#### ***Xử lý dữ liệu từ PDF và lưu vào Qdrant***

- Khai báo thư viện

In [15]:
import os
import google.generativeai as genai
import sys
sys.path.append('../')
sys.path.append("D:/Data_Platfrom/Lab/21082941_TangQuangNhatNam_Lab2/backend")

- Khởi tạo collection

In [16]:
from database import qdrant, COLLECTION_NAME
from embedding import get_google_embedding

- Kiểm tra danh sách collection được lưu trữ trong Qdrant

In [17]:
from qdrant_client import QdrantClient

client = QdrantClient("http://localhost", port=6333)
collections = client.get_collections()

print("Danh sách collections hiện có:", collections)

Danh sách collections hiện có: collections=[CollectionDescription(name='rag_lab2')]


- Hàm xử lý, đọc dữ liệu và lưu trữ vào Qdrant từ file PDF

In [18]:
import pdfplumber
import pytesseract
import uuid
import ipywidgets as widgets
from pdfminer.high_level import extract_text
from PIL import Image
from IPython.display import display
import os

def extract_text_from_pdf(pdf_path):
    """Trích xuất văn bản từ PDF bằng nhiều phương pháp."""
    text_content = []

    try:
        text = extract_text(pdf_path).strip()
        if text:
            text_content.append((text, None))  
    except Exception as e:
        print(f"Lỗi khi dùng pdfminer: {e}")

    if not text_content:
        try:
            with pdfplumber.open(pdf_path) as pdf:
                for page_num, page in enumerate(pdf.pages, start=1):
                    text = page.extract_text()
                    if text:
                        text_content.append((text.strip(), page_num))  
        except Exception as e:
            print(f"Lỗi khi dùng pdfplumber: {e}")

    if not text_content:
        try:
            with pdfplumber.open(pdf_path) as pdf:
                for page_num, page in enumerate(pdf.pages, start=1):
                    img = page.to_image()
                    img_pil = img.original
                    ocr_text = pytesseract.image_to_string(img_pil).strip()
                    if ocr_text:
                        text_content.append((ocr_text, page_num))  
        except Exception as e:
            print(f"Lỗi khi dùng OCR: {e}")

    return text_content  

def save_pdf_to_qdrant_notebook(pdf_path):
    """Lưu nội dung từ file PDF vào Qdrant với metadata."""
    extracted_data = extract_text_from_pdf(pdf_path)
    if not extracted_data:
        print(f"Không thể trích xuất văn bản từ: {pdf_path}")
        return
    
    filename = os.path.basename(pdf_path)  
    progress = widgets.IntProgress(value=0, max=len(extracted_data), description='Processing:', bar_style='info')
    display(progress)
    
    for i, (paragraph, page_num) in enumerate(extracted_data):
        embedding = get_google_embedding(paragraph)

        if embedding is None or not isinstance(embedding, list):
            print(f"Lỗi: Không thể tạo embedding cho đoạn: {paragraph[:50]}...")
            continue  

        qdrant.upsert(
            collection_name=COLLECTION_NAME,
            points=[{
                "id": str(uuid.uuid4()),
                "vector": embedding,  
                "payload": {
                    "text": paragraph,
                    "filename": filename,
                    "page_number": page_num if page_num else "Unknown"
                }
            }]
        )
        progress.value = i + 1  
    
    print(f"Đã lưu dữ liệu từ {pdf_path} vào Qdrant!")

In [19]:
save_pdf_to_qdrant_notebook("D:/Data_Platfrom/Lab/21082941_TangQuangNhatNam_Lab2/data/documents/tsla-20241023-gen-p6.pdf")

IntProgress(value=0, bar_style='info', description='Processing:', max=1)

Đã lưu dữ liệu từ D:/Data_Platfrom/Lab/21082941_TangQuangNhatNam_Lab2/data/documents/tsla-20241023-gen-p6.pdf vào Qdrant!


In [20]:
save_pdf_to_qdrant_notebook("D:/Data_Platfrom/Lab/21082941_TangQuangNhatNam_Lab2/data/documents/TuTuongHoChiMinh.pdf")

IntProgress(value=0, bar_style='info', description='Processing:', max=1)

🔹 Chia nhỏ văn bản dài 12606 bytes thành nhiều phần nhỏ...
❌ Lỗi khi tạo embedding cho đoạn: 400 Request payload size exceeds the limit: 10000 bytes.
Đã lưu dữ liệu từ D:/Data_Platfrom/Lab/21082941_TangQuangNhatNam_Lab2/data/documents/TuTuongHoChiMinh.pdf vào Qdrant!


In [21]:
results = qdrant.scroll(collection_name=COLLECTION_NAME, scroll_filter=None, limit=10)

for point in results[0]:  
    print(f"ID: {point.id}")
    print(f"Nội dung: {point.payload['text']}")
    print("="*50)

ID: 31b92b81-6e8f-42f5-8022-187a5be92ae9
Nội dung: F	I	N	A	N	C	I	A	L	S	U	M	M	A	R	Y	(Unaudited)	($	in	millions,	except	percentages	and	per	share	data)	Q3-2023	Q4-2023	Q1-2024	Q2-2024	Q3-2024	YoY	Total	automotive	revenues	19,625	21,563	17,378	19,878	20,016	2%	Energy	generation	and	storage	revenue	1,559	1,438	1,635	3,014	2,376	52%	Services	and	other	revenue	2,166	2,166	2,288	2,608	2,790	29%	Total	revenues	23,350	25,167	21,301	25,500	25,182	8%	Total	gross	profit	4,178	4,438	3,696	4,578	4,997	20%	Total	GAAP	gross	margin	17.9%	17.6%	17.4%	18.0%	19.8%	195	bp	Operating	expenses	2,414	2,374	2,525	2,973	2,280	-6%	Income	from	operations	1,764	2,064	1,171	1,605	2,717	54%	Operating	margin	7.6%	8.2%	5.5%	6.3%	10.8%	323	bp	Adjusted	EBITDA	3,758	3,953	3,384	3,674	4,665	24%	Adjusted	EBITDA	margin	16.1%	15.7%	15.9%	14.4%	18.5%	243	bp	Net	income	attributable	to	common	stockholders	(GAAP)	1,853	7,928	1,129	1,478	2,167	17%	Net	income	attributable	to	common	stockholders	(non-GAAP)	2,318	2,485	1,536	1,812	2,

In [22]:
total_points = qdrant.count(collection_name=COLLECTION_NAME)
print(f"Tổng số dữ liệu trong Collection {COLLECTION_NAME}: {total_points}")

Tổng số dữ liệu trong Collection rag_lab2: count=11


In [23]:
results = qdrant.scroll(collection_name=COLLECTION_NAME, limit=5)

for point in results[0]:
    print(f"Nội dung: {point.payload['text']}")
    print("=" * 50)

Nội dung: F	I	N	A	N	C	I	A	L	S	U	M	M	A	R	Y	(Unaudited)	($	in	millions,	except	percentages	and	per	share	data)	Q3-2023	Q4-2023	Q1-2024	Q2-2024	Q3-2024	YoY	Total	automotive	revenues	19,625	21,563	17,378	19,878	20,016	2%	Energy	generation	and	storage	revenue	1,559	1,438	1,635	3,014	2,376	52%	Services	and	other	revenue	2,166	2,166	2,288	2,608	2,790	29%	Total	revenues	23,350	25,167	21,301	25,500	25,182	8%	Total	gross	profit	4,178	4,438	3,696	4,578	4,997	20%	Total	GAAP	gross	margin	17.9%	17.6%	17.4%	18.0%	19.8%	195	bp	Operating	expenses	2,414	2,374	2,525	2,973	2,280	-6%	Income	from	operations	1,764	2,064	1,171	1,605	2,717	54%	Operating	margin	7.6%	8.2%	5.5%	6.3%	10.8%	323	bp	Adjusted	EBITDA	3,758	3,953	3,384	3,674	4,665	24%	Adjusted	EBITDA	margin	16.1%	15.7%	15.9%	14.4%	18.5%	243	bp	Net	income	attributable	to	common	stockholders	(GAAP)	1,853	7,928	1,129	1,478	2,167	17%	Net	income	attributable	to	common	stockholders	(non-GAAP)	2,318	2,485	1,536	1,812	2,505	8%	EPS	attributable	to	common	stockho

- Kiểm tra truy vấn (Query) dữ liệu đã lưu trong Qdrant

In [24]:
query_text = "Financial summary of Tesla in Q3-2023"  
query_vector = get_google_embedding(query_text)

search_results = qdrant.search(
    collection_name=COLLECTION_NAME,
    query_vector=query_vector,
    limit=5 
)

C:\Users\USER\AppData\Local\Temp\ipykernel_3020\4132296143.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_results = qdrant.search(


- Xem mức điểm số chính xác của câu truy vấn so với các embedding đã lưu trong Qdrant

In [25]:
data = [{"score": r.score, "text": r.payload["text"]} for r in search_results]
print(data)

[{'score': 0.8335724, 'text': 'F\tI\tN\tA\tN\tC\tI\tA\tL\tS\tU\tM\tM\tA\tR\tY\t(Unaudited)\t($\tin\tmillions,\texcept\tpercentages\tand\tper\tshare\tdata)\tQ3-2023\tQ4-2023\tQ1-2024\tQ2-2024\tQ3-2024\tYoY\tTotal\tautomotive\trevenues\t19,625\t21,563\t17,378\t19,878\t20,016\t2%\tEnergy\tgeneration\tand\tstorage\trevenue\t1,559\t1,438\t1,635\t3,014\t2,376\t52%\tServices\tand\tother\trevenue\t2,166\t2,166\t2,288\t2,608\t2,790\t29%\tTotal\trevenues\t23,350\t25,167\t21,301\t25,500\t25,182\t8%\tTotal\tgross\tprofit\t4,178\t4,438\t3,696\t4,578\t4,997\t20%\tTotal\tGAAP\tgross\tmargin\t17.9%\t17.6%\t17.4%\t18.0%\t19.8%\t195\tbp\tOperating\texpenses\t2,414\t2,374\t2,525\t2,973\t2,280\t-6%\tIncome\tfrom\toperations\t1,764\t2,064\t1,171\t1,605\t2,717\t54%\tOperating\tmargin\t7.6%\t8.2%\t5.5%\t6.3%\t10.8%\t323\tbp\tAdjusted\tEBITDA\t3,758\t3,953\t3,384\t3,674\t4,665\t24%\tAdjusted\tEBITDA\tmargin\t16.1%\t15.7%\t15.9%\t14.4%\t18.5%\t243\tbp\tNet\tincome\tattributable\tto\tcommon\tstockholders\t(GAAP

- Cuối cùng, ta sẽ tiến hành gửi truy vấn đến Google Gemini

In [ ]:
# Cấu hình Google AI API Key
genai.configure(api_key="GOOGLE_API_KEY")

# Lấy nội dung từ Qdrant
retrieved_texts = "\n".join([hit.payload["text"] for hit in search_results])

# Gửi prompt đến Gemini
model = genai.GenerativeModel("gemini-2.0-flash")
response = model.generate_content(f"""Dựa trên thông tin sau, hãy trả lời câu hỏi:
{retrieved_texts}
Câu hỏi: {query_text}""")

print("\nCâu trả lời từ chatbot:")
print(response.text)


Câu trả lời từ chatbot:
Dưới đây là tóm tắt tài chính của Tesla trong quý 3 năm 2023 (Q3-2023), được trích xuất từ dữ liệu được cung cấp:

*   **Total automotive revenues:** $19,625 million
*   **Energy generation and storage revenue:** $1,559 million
*   **Services and other revenue:** $2,166 million
*   **Total revenues:** $23,350 million
*   **Total gross profit:** $4,178 million
*   **Total GAAP gross margin:** 17.9%
*   **Operating expenses:** $2,414 million
*   **Income from operations:** $1,764 million
*   **Operating margin:** 7.6%
*   **Adjusted EBITDA:** $3,758 million
*   **Adjusted EBITDA margin:** 16.1%
*   **Net income attributable to common stockholders (GAAP):** $1,853 million
*   **Net income attributable to common stockholders (non-GAAP):** $2,318 million
*   **EPS attributable to common stockholders, diluted (GAAP):** $0.53
*   **EPS attributable to common stockholders, diluted (non-GAAP):** $0.66
*   **Net cash provided by operating activities:** $3,308 million
*  

- Tương tự với file TuTuongHoChiMinh.pdf, ta sẽ tiến hành thử query context và xem kết quả từ chatbot:

In [27]:
query_text = "Tư tưởng Hồ Chí Minh về giải phóng dân tộc"  
query_vector = get_google_embedding(query_text)

search_results = qdrant.search(
    collection_name=COLLECTION_NAME,
    query_vector=query_vector,
    limit=5 
)

C:\Users\USER\AppData\Local\Temp\ipykernel_3020\1670498498.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_results = qdrant.search(


In [ ]:
genai.configure(api_key="GOOGLE_API_KEY")

retrieved_texts = "\n".join([hit.payload["text"] for hit in search_results])

model = genai.GenerativeModel("gemini-2.0-flash")
response = model.generate_content(f"""Dựa trên thông tin sau, hãy trả lời câu hỏi:
{retrieved_texts}
Câu hỏi: {query_text}""")

print("\nCâu trả lời từ chatbot:")
print(response.text)


Câu trả lời từ chatbot:
Dựa vào đoạn văn đã cung cấp, tư tưởng Hồ Chí Minh về giải phóng dân tộc thể hiện ở những điểm sau:

*   **Vấn đề dân tộc là trên hết:** "Trong lúc này quyền lợi của bộ phận, của giai cấp, phải đặt dưới sự sinh tử, tồn vong của quốc gia, của dân tộc. Trong lúc này, nếu không giải quyết được vấn đề dân tộc giải phóng, không đòi được độc lập, tự do cho toàn thể dân tộc thì chẳng những toàn thể quốc gia dân tộc còn chịu mãi kiếp ngựa trâu, mà quyền lợi của bộ phận, giai cấp đến vạn năm cũng không đòi lại được”.
*   **Đại đoàn kết dân tộc:** Tập hợp mọi lực lượng yêu nước, không phân biệt giai cấp, tôn giáo, thành phần, để chống kẻ thù chung (phát xít Nhật - Pháp).
*   **Chủ trương thành lập Mặt trận Dân tộc Thống nhất:** Cụ thể là Mặt trận Việt Minh, để đoàn kết toàn dân tộc.
*   **Giải quyết vấn đề dân tộc trong khuôn khổ mỗi nước Đông Dương:** Việt Nam, Lào, Campuchia liên hệ mật thiết, giúp đỡ nhau giành thắng lợi.
*   **Khởi nghĩa vũ trang:** Phương pháp cách 